In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from QuantLib import *

plt.style.use('fivethirtyeight')

In [33]:
start_date = Date(30, 4, 2017)
maturity_date = Date(31, 7, 2017)
freq = Period(1, Days)
option_type = Option.Put
strike_price = 1.
spot_price = 1.
volatility = 0.41

risk_free_rate = 0.035
dividend_rate = 0.
day_count = Actual365Fixed()
calendar = China(China.SSE)

payoff = PlainVanillaPayoff(option_type, strike_price)
exercise = EuropeanExercise(maturity_date)

In [46]:
fixing_start_date = Date(30, 4, 2017)

fixing_scheduler = Schedule(fixing_start_date, maturity_date, freq, calendar, Following, Following, DateGeneration.Backward, False)
fixing_dates = [d for d in fixing_scheduler][1:]

In [47]:
calculation_date = start_date
Settings.instance().evaluationDate = calculation_date

In [48]:
spot_handle = RelinkableQuoteHandle(SimpleQuote(spot_price))
flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, volatility, day_count))

bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
option = DiscreteAveragingAsianOption(Average.Arithmetic, 0., 0, fixing_dates, payoff, exercise)

In [49]:
engine = FdBlackScholesAsianEngine(bsm_process, 200, 100, 100)
option.setPricingEngine(engine)
fdm_price = option.NPV()
fdm_delta = option.delta()
print("BSM ASIAN FDM price is {0:.4f}".format(fdm_price))
print("BSM ASIAN FDM delta is {0:.4f}".format(fdm_delta))

BSM ASIAN FDM price is 0.0463
BSM ASIAN FDM delta is -0.4635


In [50]:
option2 = EuropeanOption(payoff, exercise)
engine2 = AnalyticEuropeanEngine(bsm_process)
option2.setPricingEngine(engine2)
price2 = option2.NPV()
delta2 = option2.delta()
print("BSM European price is {0:.4f}".format(price2))
print("BSM European delta is {0:.4f}".format(delta2))

BSM European price is 0.0773
BSM European delta is -0.4420


In [ ]:
option2 = EuropeanOption